# Grating Reduced Basis
\begin{equation}
    n \lambda = d \sin(\alpha)
\end{equation}
where $n = 1$, $d = \frac{1}{s}$ and $\lambda = \frac{c2\pi}{\omega}$
\begin{equation}
    \alpha = \arcsin\big(\frac{2\pi s}{\omega}\big)
\end{equation}

In [11]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from ngsolve import *
from netgen.geom2d import unit_square


import numpy as np
import scipy
import math
import time
from math import pi

from ngsolve.webgui import Draw
from reduced_basis_myQR import *


In [12]:
def ChebPoints(a,b,n):
    x_i = []
    for i in range(0,n+1):
        fac = (2*i+1)/(2*n+2)
        x_ch = np.cos(pi*fac)
        x_i.append((a+b)/2+((b-a)/2)*x_ch)
    return x_i

In [13]:
def setang(alpha,d):
    if alpha > 90 or alpha < 30:
        return print("wrong angle")
    return d/(np.sin(np.radians(alpha)))

maxind = 50
mode = "Cheb"
s = 40

# set the minimumangle
alpha1 = 30
# set the maxangle
alpha2 = 70

lower = setang(alpha2,s)
upper = setang(alpha1,s)
if mode == "Cheb":
    omegfact = np.sort(ChebPoints(lower,upper,maxind))
if mode == "equi":
    omegfact = np.linspace(lower,upper,maxind)
snapshots = []
for i in range(maxind):
    snapshots.append(2*math.pi*omegfact[i])

In [16]:
from netgen.geom2d import *

geo = SplineGeometry()
geo.AddRectangle ( (0,0), (1,1), bcs=["bottom","right","top","left"], leftdomain=1)
r = 0.005
for i in range(s-1):
    geo.AddCircle ( (0.5,(i+1)/(s)), r=r, bc="interface", 
                     leftdomain=0, rightdomain=1)
geo.SetMaterial(1, "medium")
mesh = Mesh(geo.GenerateMesh(maxh=0.02))
mesh.Curve(3)
Draw(mesh)

fes = H1(mesh, order=5, complex=True, dirichlet="interface")
n = { "medium" : 1, "dot" : 3 }
cfn = CoefficientFunction( [n[mat] for mat in mesh.GetMaterials()] )

source = 80*exp(-20**2*(y-0.5)*(y-0.5))
space = ReducedBasis(fes, source,cfn)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2006-98-ge712f407', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, …

In [17]:
space.setSnapshots(snapshots)
slider_func1 = lambda alpha: space.draw((2*pi*s)/(np.sin(np.radians(alpha))),redraw=True)
space.draw(space.omega_max,minval = -0.00007,maxval = 0.00007,autoscale = False)
interact(slider_func1, alpha=widgets.FloatSlider(min=alpha1, max=alpha2, step=1, value=alpha1))

set snapshots and reset basis
compute Reduced Basis
Calculate QR_Decomposition
QR-calculation Time:  21.10588049888611 s
finished computing Reduced Basis
omega: 502.15316844427724, norm of solution: (1.3138223854094102e-08+1.5933616074490894e-27j)


NGSWebGuiWidget(value={'ngsolve_version': '6.2.2006-98-ge712f407', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, …

interactive(children=(FloatSlider(value=30.0, description='alpha', max=70.0, min=30.0, step=1.0), Output()), _…

<function __main__.<lambda>(alpha)>

In [ ]:
random_omegas = np.sort(np.append(np.random.uniform(space.omega_min, space.omega_max, 100), snapshots))
%matplotlib notebook
import matplotlib.pyplot as plt

space.logging = False

norm_random_omegas, residual_random_omegas = space.computeValues(random_omegas, cheap = False)

print(" average residual: {} \n amount of snapshots: {}".format(np.mean(residual_random_omegas), len(space.getSnapshots())))

fig, ax = plt.subplots()
ax.plot(random_omegas, norm_random_omegas,  '-', label = "norm")
ax.set_xlabel("omega")
ax.set_ylabel("norm of solution")
ax.set_title("norm of solution in reduced space")
plt.semilogy()
plt.savefig("norm of solution in reduced space {} {}".format(mode,maxind))

fig, ax = plt.subplots()         
ax.plot(random_omegas, residual_random_omegas , '-',label = "average residual: {}".format(np.mean(residual_random_omegas)))
sn_residual = space.computeValues(space.getSnapshots(), norm= False, cheap = False)
ax.plot(space.getSnapshots(), sn_residual, 'r.', label = "snapshot parameters")
ax.legend()
ax.set_xlabel("omega")
ax.set_ylabel('residual of solution')
ax.set_title("residual in reduced space")
plt.semilogy()
plt.savefig("residual in reduced space {} {}".format(mode,maxind))